<a href="https://colab.research.google.com/github/kiboiWanjohi/scotep-kenya/blob/main/histopathology/histopathology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp /content/drive/MyDrive/Stat_Docs/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
# Change permission
! chmod 600 ~/.kaggle/kaggle.json

### Download Histopathology Dataset

Link To Breast Histopathology Dataset [Here](https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images)

In [ ]:
! kaggle datasets download -d paultimothymooney/breast-histopathology-images"
      

In [ ]:
# unzip content 
! unzip /content/breast-histopathology-images.zip -d /content/breast-histopathology-images

In [ ]:
# check disk usage 
! df -h 

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import os
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import random
import warnings
import PIL
import random
import plotly.express as px

In [ ]:
image_dir = 'content/jpeg'